<a href="https://colab.research.google.com/github/HaJunYoo/Inflearn_RecSys/blob/main/2%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MovieLens 100K 데이터는 3가지 파일로 구성
- 사용자 데이터 : u.user
- 영화에 대한 데이터 : u.item
- 영화 평가 데이터 : u.data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pwd

/content


In [6]:
import os
import pandas as pd

base_src = '/content/drive/MyDrive/추천시스템/RecoSys/Data'
u_user_src = os.path.join(base_src, 'u.user')
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']

users = pd.read_csv( u_user_src, 
                    sep = '|' ,
                    names = u_cols,
                    encoding = 'latin-1'
                    )

users = users.set_index('user_id')
users.head()

,age,sex,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


In [7]:
# u.item을 DataFrame
u_item_src = os.path.join(base_src, 'u.item')
i_cols = ['movie_id','title','release date','video release date',
'IMDB URL','unknown','Action','Adventure','Animation',
'Children\'s','Comedy','Crime','Documentary','Drama','Fantasy',
'FilmNoir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']

movies = pd.read_csv(u_item_src,
                     sep = '|',
                     names = i_cols,
                     encoding='latin-1')
movies = movies.set_index('movie_id')
movies.head()


,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,FilmNoir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [17]:
u_data_src = os.path.join(base_src, 'u.data')
r_cols = ['user_id', 'movie_id','rating','timestamp']
ratings = pd.read_csv(u_data_src,
                      sep = '\t',
                      names = r_cols,
                      encoding = 'latin-1')


In [19]:
ratings = ratings.set_index('user_id')
ratings.head()

,movie_id,rating,timestamp
user_id,,,
196,242,3,881250949
186,302,3,891717742
22,377,1,878887116
244,51,2,880606923
166,346,1,886397596


In [20]:
# 인기 제품 추천 방식 function
def recom_movie(n_items):
  movie_mean = ratings.groupby(['movie_id'])['rating'].mean()
  movie_sort = movie_mean.sort_values(ascending=False)[:n_items]
  recom_movies = movies.loc[movie_sort.index]
  recommendations = recom_movies['title']
  return recommendations

recom_movie(5)

movie_id
814                         Great Day in Harlem, A (1994)
1599                        Someone Else's America (1995)
1201           Marlene Dietrich: Shadow and Light (1996) 
1122                       They Made Me a Criminal (1939)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object

In [21]:
ratings.groupby(['movie_id'])['rating'].mean()

movie_id
1       3.878319
2       3.206107
3       3.033333
4       3.550239
5       3.302326
          ...   
1678    1.000000
1679    3.000000
1680    2.000000
1681    3.000000
1682    3.000000
Name: rating, Length: 1682, dtype: float64

In [22]:
# 100k 영화 평점에 대해서 실제 값과 best-seller 방식으로 구한 예측값의 rmse 를 계산

import numpy as np

def RMSE(y_true, y_pred):
  return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))


# 정확도 계산
rmse = []
movie_mean = ratings.groupby(['movie_id'])['rating'].mean()

for user in set(ratings.index):
  y_true = ratings.loc[user]['rating']
  # best-seller 방식으로
  y_pred = movie_mean[ratings.loc[user]['movie_id']]
  accuracy = RMSE(y_true, y_pred)
  rmse.append(accuracy)


# RMSE 계산
print(np.mean(rmse))


0.996007224010567


In [52]:
# 데이터 불러오기
import os
import pandas as pd

base_src = '/content/drive/MyDrive/추천시스템/RecoSys/Data'
u_user_src = os.path.join(base_src, 'u.user')
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']

users = pd.read_csv( u_user_src, 
                    sep = '|' ,
                    names = u_cols,
                    encoding = 'latin-1'
                    )

# users = users.set_index('user_id')

# u.item을 DataFrame
u_item_src = os.path.join(base_src, 'u.item')
i_cols = ['movie_id','title','release date','video release date',
'IMDB URL','unknown','Action','Adventure','Animation',
'Children\'s','Comedy','Crime','Documentary','Drama','Fantasy',
'FilmNoir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']

movies = pd.read_csv(u_item_src,
                     sep = '|',
                     names = i_cols,
                     encoding='latin-1')
# movies = movies.set_index('movie_id')

# u_data
u_data_src = os.path.join(base_src, 'u.data')
r_cols = ['user_id', 'movie_id','rating','timestamp']
ratings = pd.read_csv(u_data_src,
                      sep = '\t',
                      names = r_cols,
                      encoding = 'latin-1')
# ratings = ratings.set_index('user_id')


In [53]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [54]:
# rating dataframe에서 timestamp 제거
ratings = ratings.drop('timestamp', axis=1)


In [55]:
movies.head()

,movie_id,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,FilmNoir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [56]:
movies = movies[['movie_id','title']]


In [57]:
# 데이터 train, test set 분리
from sklearn.model_selection import train_test_split

x = ratings.copy()
y = ratings['user_id']

x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size = 0.25,
                                                    # 층화 추출  
                                                    # 라벨 비율 반영해서 추출 
                                                    stratify = y)


# 정확도를 계산하는 함수
import numpy as np

def RMSE(y_true, y_pred):
  return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 모델별 RMSE를 계산하는 함수
def score(model):
  id_pairs = zip(x_test['user_id'],x_test['movie_id'])
  # model에 user과 movie를 반복적으로 넣으면서 예측 결과 array를 구성
  y_pred = np.array([model(user,movie) for (user,movie) in id_pairs])
  y_true = np.array(x_test['rating'])
  return RMSE(y_true,y_pred)


train_mean = x_train.groupby(['movie_id'])['rating'].mean()
def best_seller(user_id,movie_id):
  try:
    rating= train_mean[movie_id]
  except:
    rating = 3.0
  return rating

score(best_seller)
# test 셋에 결과를 내서 성능이 낮음

1.027912782094389

In [58]:
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [59]:
x_train.head()

,user_id,movie_id,rating
37792,295,559,4
88616,825,456,3
8005,123,50,3
65843,840,474,5
60112,655,860,3


In [60]:
# 성별에 따른 예측값 계산
merged_ratings = pd.merge(x_train, users)

merged_ratings.head()

,user_id,movie_id,rating,age,sex,occupation,zip_code
0,295,559,4,31,M,educator,50325
1,295,22,4,31,M,educator,50325
2,295,403,4,31,M,educator,50325
3,295,227,4,31,M,educator,50325
4,295,69,5,31,M,educator,50325


In [61]:
users = users.set_index('user_id')

g_mean = merged_ratings[['movie_id','sex','rating']].groupby(['movie_id','sex'])['rating'].mean()

g_mean

movie_id  sex
1         F      3.835052
          M      3.921875
2         F      3.600000
          M      3.206897
3         F      2.916667
                   ...   
1678      M      1.000000
1679      M      3.000000
1680      M      2.000000
1681      M      3.000000
1682      M      3.000000
Name: rating, Length: 3040, dtype: float64

In [62]:
# movie_id, user_id full matrix 구성
rating_matrix = x_train.pivot(
    index = 'user_id',
    columns = 'movie_id',
    values = 'rating'
)

rating_matrix

movie_id,1,2,3,4,5,6,7,8,9,10,...,1672,1673,1674,1675,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,3.0,NaN,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
# Gender 기준 추천
# movie id가 user의 성별을 가져옴
# 예측대상 결과가 성별 평균을 담고 있는지 없다면 3.0
# movie_id가 없으면 그냥 3.0
def cf_gender(user_id, movie_id):
  if movie_id in rating_matrix.columns:
    gender = users.loc[user_id]['sex']
    if gender in g_mean[movie_id].index:
      gender_rating = g_mean[movie_id][gender]
    else:
      gender_rating = 3.0
  else:
    gender_rating = 3.0
  return gender_rating

score(cf_gender)
# gender 기준 추천은 크게 도움이 되지 않음

1.0373907448295734

In [65]:
gender = users.loc[940]['sex']
gender

'M'

In [67]:
gender in g_mean[1672].index

True